#Predictive Failure Modelling Simulation

##Configs

In [ ]:
#Connecting to G Drive
from google.colab import drive

drive.mount('/gdrive')
%cd /gdrive/MyDrive/ColabNotebooks/PredictiveModelling/data/

Mounted at /gdrive
/gdrive/MyDrive/PredictiveModelling/data


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn import metrics

In [ ]:
all_data_df = pd.read_csv('combined_data.csv', index_col='Unnamed: 0')
all_data_df['datetime'] = pd.to_datetime(all_data_df['datetime']) #no formatting needed as it is already default style

In [ ]:
all_data_df.head()

,datetime,machineID,volt,rotate,pressure,vibration,comp1,comp2,comp3,comp4,error1,error2,error3,error4,error5,failure
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,0.041667,0.041667,0.041667,0.041667,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,0.083333,0.083333,0.083333,0.083333,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,0.125000,0.125000,0.125000,0.125000,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,0.166667,0.166667,0.166667,0.166667,NaN,NaN,NaN,NaN,NaN,NaN


##Model to be used

In [ ]:
MAX_DEPTH = 4         #depth of trees
N_ESTIMATORS = 300    #number of trees
LEARNING_RATE = 0.08

In [ ]:
model = XGBClassifier(
    silent = True,
    max_depth = MAX_DEPTH,
    n_estimators = N_ESTIMATORS,
    learning_rate = LEARNING_RATE,
    tree_method = 'hist',
    seed=8
)

##Functions to get necessary stats from TS data

In [ ]:
tel_columns = ["volt", "rotate", "pressure", "vibration"]
error_columns = ["error1", "error2", "error3", "error4", "error5"]
maint_columns = ["comp1", "comp2", "comp3", "comp4"]

In [ ]:
def validate_machine(df, lag_start, lag_length):
  if df['machineID'][lag_start-lag_length] == df['machineID'][lag_start]:
    return True
  else:
    return False

In [ ]:
def validate_datetime(df, lag_start, lag_length):
  if df['datetime'][lag_start-lag_length]+pd.Timedelta(str(lag_length)+' hours') == df['datetime'][lag_start]:
    return True
  else:
    return False

In [ ]:
def get_stats(df, lag_start, lag_length):
  lag_df = df.iloc[lag_start-lag_length:lag_start+1]
  
  if not validate_machine(lag_df, lag_start, lag_length):
    raise ValueError('This DataFrame consists of data across different machines at index {} for {} hours of lag'.format(lag_start,lag_length))
  
  if not validate_datetime(lag_df, lag_start, lag_length):
    raise ValueError('The DataFrame does not contain {} hours of continuous data at index {}'.format(lag_length, lag_start))
  
  res = []
  for c in tel_columns:
    res.extend([
                lag_df[c].min(),
                lag_df[c].max(),
                lag_df[c].std(),
                lag_df[c].mean()
    ])
  
  for c in error_columns:
    res.append(lag_df[c].sum())
  
  for c in maint_columns:
    res.append(lag_df[c].iloc[-1])

  return res

In [ ]:
#create column titles for statistics dataframe
col_titles = ['datetime']
for c in tel_columns:
  col_titles.append(c+'_min')
  col_titles.append(c+'_max')
  col_titles.append(c+'_std')
  col_titles.append(c+'_mean')
col_titles.extend(error_columns)
col_titles.extend(maint_columns)

for c in col_titles.copy():
  if c != 'datetime':
    col_titles.append(c+'3')

##Simulation Setup

In [ ]:
dti = pd.date_range(str(all_data_df['datetime'][0]+pd.Timedelta('7 days')), periods=51, freq="7D")
dti[0:3]

DatetimeIndex(['2015-01-08 06:00:00', '2015-01-15 06:00:00',
               '2015-01-22 06:00:00'],
              dtype='datetime64[ns]', freq='7D')

In [ ]:
all_data_df['datetime'][[0,len(all_data_df)-1]]

0        2015-01-01 06:00:00
876141   2016-01-01 06:00:00
Name: datetime, dtype: datetime64[ns]

###Function to create augmented dataframe based on selected date

In [ ]:
def get_augmented_df(df, date_string):
  augmented_df = df[df['datetime']<=date_string].reset_index(drop=True)

  return augmented_df

###Function to capture machine failure statistics

In [ ]:
def get_failure_statistics_df(df):
  lag_length = 24
  lag_length_short = 3
  predict_ahead = 12
  
  failures_indx = df.index[df['failure'].notnull()]
  
  failures_stats = []

  for f in failures_indx:
    try:
      temp_ls = [df['datetime'][f-predict_ahead]]
      temp_ls.extend(get_stats(df, f-predict_ahead, lag_length))
      temp_ls.extend(get_stats(df, f-predict_ahead, lag_length_short))
      failures_stats.append(temp_ls)
    except Exception as e:
      None #print(e)
  
  failures_stats_df = pd.DataFrame(failures_stats, columns = col_titles)
  failures_stats_df['failure'] = True

  return failures_stats_df

###Function to create DataFrame with normal functioning machinery data

In [ ]:
def get_normal_statistics_df(df):
  lag_length = 24
  lag_length_short = 3
  predict_ahead = 12
  
  failures_indx = df.index[df['failure'].notnull()]

  failure_indexes_plus = []
  for f in failures_indx:
    failure_indexes_plus.extend(np.arange(f-36,f+1))
  
  normal_indexes = df.drop(failure_indexes_plus).sample(len(df)//80, random_state=8).index

  normal_stats = []

  for n in normal_indexes:
    try:
      temp_ls = [df['datetime'][f-predict_ahead]]
      temp_ls.extend(get_stats(df, n-predict_ahead, lag_length))
      temp_ls.extend(get_stats(df, n-predict_ahead, lag_length_short))
      normal_stats.append(temp_ls)
    except Exception as e:
      None #print(e)

  normal_stats_df = pd.DataFrame(normal_stats, columns = col_titles)
  normal_stats_df['failure'] = False

  return normal_stats_df

###Function to combine DataFrames

In [ ]:
def combine_dataframes(failures, normals):
  total_stats_df = pd.concat([failures, normals], ignore_index=True)

  #shuffle around DataFrame
  total_stats_df = total_stats_df.sample(len(total_stats_df), random_state=8)

  return total_stats_df

###Function to split data

In [ ]:
def split_df(df):
  split_mask = np.random.rand(len(df)) < 0.8

  x_df = df.drop(['datetime','failure'], axis=1)
  y_df = df['failure']

  x_train = x_df[split_mask]
  y_train = y_df[split_mask]

  x_validation = x_df[~split_mask]
  y_validation = y_df[~split_mask]

  return x_train, y_train, x_validation, y_validation

###Function to train model

In [ ]:
def train_model(model, x_train, y_train, x_validation, y_validation):
  model.fit(
    x_train, 
    y_train, 
    eval_set=[(x_validation, y_validation)], 
    early_stopping_rounds=8, 
    eval_metric="aucpr"
  )

  return model

###Function to test model

In [ ]:
def test_model(model, df, old_date):
  temp_df = df[df['datetime']>old_date].reset_index(drop=True)

  failure_df = get_failure_statistics_df(temp_df)
  normal_df = get_normal_statistics_df(temp_df)
  statistics_df = combine_dataframes(failure_df, normal_df)
  
  y_df = statistics_df['failure']
  x_df = statistics_df.drop(['datetime', 'failure'], axis=1)

  test_predictions = model.predict(x_df)
  precision, recall, fscore, _ = metrics.precision_recall_fscore_support(y_df, test_predictions, average='weighted')
  confusion_matrix = metrics.confusion_matrix(y_df, test_predictions)

  return precision, recall, fscore, confusion_matrix

##Simulation

In [ ]:
#31 min 15s

simulation_data = []
error_log = ''

for indx, date in enumerate(dti):
  print('Starting week {}\n'.format(indx+1))

  df = get_augmented_df(all_data_df, date)
  failure_df = get_failure_statistics_df(df)
  normal_df = get_normal_statistics_df(df)
  statistics_df = combine_dataframes(failure_df, normal_df)

  x_train, y_train, x_validation, y_validation = split_df(statistics_df)
  
  if indx != 0:
    try:
      precision, recall, fscore, confusion_matrix = test_model(model, df, dti[indx-1])

      pos_ratio = confusion_matrix[0][1]/confusion_matrix[1][1]
      false_pos = confusion_matrix[1][0]/confusion_matrix[1].sum()

      simulation_data.append(['Week {}'.format(indx+1), training_points, precision, recall, fscore, confusion_matrix, pos_ratio, false_pos])
    except Exception as e:
      error_log += 'Error:"' + str(e) + '" at week {}'.format(indx+1) + '\n'

  model = train_model(model, x_train, y_train, x_validation, y_validation)
  
  training_points = len(x_train)
  
  print('\nDone with week {}'.format(indx+1))

Starting week 1

[0]	validation_0-aucpr:0.5
Will train until validation_0-aucpr hasn't improved in 8 rounds.
[1]	validation_0-aucpr:0.5
[2]	validation_0-aucpr:0.5
[3]	validation_0-aucpr:0.5
[4]	validation_0-aucpr:0.5
[5]	validation_0-aucpr:0.5
[6]	validation_0-aucpr:0.5
[7]	validation_0-aucpr:1
[8]	validation_0-aucpr:1
[9]	validation_0-aucpr:1
[10]	validation_0-aucpr:1
[11]	validation_0-aucpr:1
[12]	validation_0-aucpr:1
[13]	validation_0-aucpr:1
[14]	validation_0-aucpr:1
[15]	validation_0-aucpr:1
Stopping. Best iteration:
[7]	validation_0-aucpr:1


Done with week 1
Starting week 2

[0]	validation_0-aucpr:0.57172
Will train until validation_0-aucpr hasn't improved in 8 rounds.
[1]	validation_0-aucpr:0.57172
[2]	validation_0-aucpr:0.57172
[3]	validation_0-aucpr:0.57172
[4]	validation_0-aucpr:0.57172
[5]	validation_0-aucpr:0.57172
[6]	validation_0-aucpr:0.780278
[7]	validation_0-aucpr:0.780278
[8]	validation_0-aucpr:0.780278
[9]	validation_0-aucpr:0.780278
[10]	validation_0-aucpr:0.780278

In [ ]:
print(error_log)

simulation_column_titles = ['week', 'number_training', 'precision', 'recall', 'fscore', 'confusion_matrix', 'pos_ratio', 'false_pos_rate']

simulation_stats_df = pd.DataFrame(simulation_data, columns = simulation_column_titles)
simulation_stats_df

Error:"'[-13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1] not found in axis'" at week 10
Error:"'[-13 -12 -11 -10  -9  -8  -7  -6  -5  -4  -3  -2  -1] not found in axis'" at week 25



,week,number_training,precision,recall,fscore,confusion_matrix,pos_ratio,false_pos_rate
0,Week 2,162,0.956275,0.961326,0.957626,"[[170, 2], [5, 4]]",0.500000,0.555556
1,Week 3,347,0.953267,0.952128,0.952640,"[[163, 5], [4, 16]]",0.312500,0.200000
2,Week 4,532,0.950667,0.933333,0.939750,"[[172, 10], [3, 10]]",1.000000,0.230769
3,Week 5,703,0.923744,0.932642,0.924568,"[[173, 3], [10, 7]]",0.428571,0.588235
4,Week 6,874,0.976258,0.974093,0.975055,"[[184, 3], [2, 4]]",0.750000,0.333333
5,Week 7,1063,0.978758,0.979275,0.978685,"[[175, 1], [3, 14]]",0.071429,0.176471
6,Week 8,1237,0.991940,0.989637,0.990257,"[[184, 2], [0, 7]]",0.285714,0.000000
7,Week 9,1412,0.989744,0.989744,0.989744,"[[181, 1], [1, 12]]",0.083333,0.076923
8,Week 11,1755,0.971841,0.972527,0.972095,"[[165, 2], [3, 12]]",0.166667,0.200000
9,Week 12,1928,0.990106,0.990000,0.989642,"[[186, 0], [2, 12]]",0.000000,0.142857


##Output data as csv for visualizations

In [ ]:
path = %pwd

In [ ]:
with open(path+'/simulation_data.csv', 'w', encoding = 'utf-8-sig') as f:
  simulation_stats_df.to_csv(f)